In [1]:
import pandas as pd
import re
import translators.server as tss
TARGET_LANG = 'es'

Using state Pennsylvania server backend.


In [2]:
# read the latest mass production table
# for 7.17
# path_new = "./Output_cleaned_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalMassProduction_new_cleaned.txt"
# path_ref = './Greg_table_folder/7.17 - Sheet1.csv'
# path_output = './Output_translated_folder/7.17 HTML/7.17 HTML/MassProduction/7.17_translated.txt'
# path_no_mark = './Output_translated_folder/7.17 HTML/7.17 HTML/MassProduction/7.17_translated_no_mark.txt'

# for 7.10
path_new = "./Output_cleaned_folder/7.10 HTML/7.10 HTML/MassProduction/7_10_cleaned.txt"
path_ref = './Greg_table_folder/7.10 - Sheet1.csv'
path_output = './Output_translated_folder/7.10 HTML/7.10 HTML/MassProduction/7.10_translated.txt'
path_no_mark = './Output_translated_folder/7.10 HTML/7.10 HTML/MassProduction/7.10_translated_no_mark.txt'

# for 6.05
# path_new = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/MassProduction/6_5_cleaned.txt"
# path_ref = './Greg_table_folder/6.05 - Sheet1.csv'
# path_output = './Output_translated_folder/6.05 HTML/6.05 HTML/MassProduction/6.05_translated.txt'
# path_no_mark = './Output_translated_folder/6.05 HTML/6.05 HTML/MassProduction/6.05_translated_no_mark.txt'

In [3]:
table_clean = pd.read_csv(path_new, sep="\t", index_col=0, keep_default_na=False)

In [4]:
# Change column name
table_clean_col_list = []
for i in table_clean:
    table_clean_col_list.append(i)
    table_clean_col_list.append(i+"ESP")
len(table_clean_col_list)

# Create a new table, with the column from the Greg's mass production table and the index from the latest mass production table
table_translated = pd.DataFrame(columns=table_clean_col_list, index=table_clean.index)
table_translated_clean = pd.DataFrame(columns=table_clean_col_list, index=table_clean.index)
table_translated

,01negativeslope,01negativeslopeESP,02negativeslope,02negativeslopeESP,03negativeslope,03negativeslopeESP,04negativeslope,04negativeslopeESP,05negativeslope,05negativeslopeESP,06negativeslope,06negativeslopeESP,07negativeslope,07negativeslopeESP,08negativeslope,08negativeslopeESP,09negativeslope,09negativeslopeESP
Problem Name,,,,,,,,,,,,,,,,,,
%(startStateNodeName)%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
%(feedbackmode)%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
%(problem_statement)%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
%(problem_info)%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
%(Input_80_xpoint4xypoint4y)%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
%(Input_88_curveneedsmorepoints)%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
%(Input_114_slope_x_initial_value)%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# read the Greg's mass production table
table_old = pd.read_csv(path_ref, header = None)
# table_old.head()
# skip some rows, because they are translation for HTML elements
header_index = table_old.index[table_old[0] == 'Problem Name'].to_list()
header_index
# reload the csv
table_old = pd.read_csv(path_ref, header = header_index)
table_old.columns
# table_old = None

Index(['Problem Name', 'Problem1', 'Problem1ESP', 'Problem Name.1', 'Problem2',
       'Problem2ESP', 'Problem Name.2', 'Problem3', 'Problem3ESP',
       'Problem Name.3', 'Problem4', 'Problem4ESP', 'Problem Name.4',
       'Problem5', 'Problem5ESP', 'Problem Name.5', 'Problem6', 'Problem6ESP',
       'Problem Name.6', 'Problem7', 'Problem7ESP', 'Problem Name.7',
       'Problem8', 'Problem8ESP', 'Problem Name.8', 'Problem9', 'Problem9ESP'],
      dtype='object')

In [6]:
def replace_variable(content_new, column):
    pattern_variable = "%\((.*?)\)%"
    # count the number of the replacement in one variable(content_new)
    match_variable = [match for match in re.findall(pattern_variable, str(content_new))]
    for i in range(len(match_variable)):
        # find the variable
        variable = "%(" + str(match_variable[i]) + ")%"
        # find the corresponding column name, and then find the value
        column_name = table_clean.columns[column]
        try:
            value = table_clean.loc[variable, column_name] 
            # print(variable, value)
            content_new = content_new.replace(variable, value)
        except:
            print(column_name, variable + " doesn't exist")
    return content_new

In [7]:
def replace_string(formula, column, replacment="*"):
    pattern_string = '"(.*?)"'
    # string_dic = {}
    match_string = [match for match in re.findall(pattern_string, formula)]
    # print("old", formula)
    if match_string:
        for i in range(len(match_string)):
            string = str(match_string[i])
            # print(string, replace_variable(string, column))
            formula = formula.replace(string, replace_variable(string, column)) 
    # print("new", formula)  
    return formula

In [8]:
def replace_formula(content_new, column, replacement="#"):
    replacement_sign = replacement
    replacement_formula_dic = {}
    pattern_formula = "<%(.*?)%>"
    # count the number of the replacement in one variable(content_new)
    match_formula = [match for match in re.findall(pattern_formula, str(content_new))]
    if match_formula:
        for i in range(len(match_formula)):
            old_formula = match_formula[i]
            if '"' in old_formula:
                formula = replace_string(old_formula, column)
                final_formula = str(formula)
            else:
                final_formula = str(old_formula)
            replacement = replacement_sign + str(i)
            replacement_formula_dic[replacement] = final_formula
            content_new = content_new.replace(old_formula, replacement)
    return content_new, replacement_formula_dic

In [9]:
def translate_string(formula, replacement="*"):
    pattern_string = '"(.*?)"'
    # string_dic = {}
    match_string = [match for match in re.findall(pattern_string, formula)]
    # print("old", formula)
    if match_string:
        for i in range(len(match_string)):
            string = str(match_string[i])
            # print(string, replace_variable(string, column))
            try:
                translation = tss.google(string, from_language='en', to_language=TARGET_LANG)
            except:
                translation = 'error'
            formula = formula.replace(string, translation) 
    # print("new", formula)  
    return formula

In [13]:
def translate_formula(content_new, replacement="#"):
    replacement_sign = replacement
    translation_formula_dic = {}
    pattern_formula = "<%(.*?)%>"
    # count the number of the replacement in one variable(content_new)
    match_formula = [match for match in re.findall(pattern_formula, str(content_new))]
    if match_formula:
        for i in range(len(match_formula)):
            old_formula = match_formula[i]
            if '"' in old_formula:
                print(old_formula)
                formula = translate_string(old_formula)
                final_formula = str(formula)
            else:
                final_formula = str(old_formula)
            replacement = replacement_sign + str(i)
            translation_formula_dic[replacement] = final_formula
            content_new = content_new.replace(old_formula, replacement)
    return content_new, translation_formula_dic

In [39]:
formula = '=%(point1x)% > graphMaxX ? "adjust the right boundary by changing its value to higher than 1, then " : ""'

pattern_string = '"(.*?)"'
# string_dic = {}
match_string = [match for match in re.findall(pattern_string, formula)]
print("old", formula)
print(match_string)
if match_string:
    for i in range(len(match_string)):
        string = str(match_string[i])
        if string == '':
            pass
            print(string)
        else:
            try:
                translation = tss.google(string, from_language='en', to_language=TARGET_LANG)
            except:
                translation = 'error'
            formula = formula.replace(string, translation) 
formula

old =%(point1x)% > graphMaxX ? "adjust the right boundary by changing its value to higher than 1, then " : ""
['adjust the right boundary by changing its value to higher than 1, then ', '']



'=%(point1x)% > graphMaxX ? "Ajuste el límite derecho cambiando su valor a más de 1, luego" : ""'

In [29]:
content_new = 'Please <%=%(point1x)% > graphMaxX ? "adjust the right boundary by changing its value to higher than 1, then " : ""%><%=graphMinX > %(point1x)% ? "adjust the left boundary by lowering it to less than 1, then " : ""%><%=%(point1y)% > graphMaxY ? "adjust the upper boundary by increasing its value to more than 36, then " : ""%><%=graphMinY > %(point1y)% ? "change the lower boundary by decreasing it to less than 36, then " : ""%>add the point by clicking on the right spot in the graph.'

In [15]:
translate_formula(content_new)

=%(point1x)% > graphMaxX ? "adjust the right boundary by changing its value to higher than 1, then " : ""
=graphMinX > %(point1x)% ? "adjust the left boundary by lowering it to less than 1, then " : ""
=%(point1y)% > graphMaxY ? "adjust the upper boundary by increasing its value to more than 36, then " : ""
=graphMinY > %(point1y)% ? "change the lower boundary by decreasing it to less than 36, then " : ""


('Please <%#0%><%#1%><%#2%><%#3%>add the point by clicking on the right spot in the graph.',
 {'#0': 'error=error%error(errorperroroerrorierrornerrorterror1errorxerror)error%error error>error errorgerrorrerroraerrorperrorherrorMerroraerrorxerrorXerror error?error error"errorAerrorjerroruerrorserrorterroreerror erroreerrorlerror errorlerroríerrormerrorierrorterroreerror errorderroreerrorrerroreerrorcerrorherroroerror errorcerroraerrormerrorberrorierroraerrornerrorderroroerror errorserroruerror errorverroraerrorlerroroerrorrerror erroraerror errormerroráerrorserror errorderroreerror error1error,error errorlerroruerroreerrorgerroroerror"error error:error error"error"error',
  '#1': 'error=errorgerrorrerroraerrorperrorherrorMerrorierrornerrorXerror error>error error%error(errorperroroerrorierrornerrorterror1errorxerror)error%error error?error error"errorAerrorjerroruerrorserrorterroreerror erroreerrorlerror errorlerroríerrormerrorierrorterroreerror errorierrorzerrorqerroruerrorierroreerror

In [16]:
print(content_new)

Please <%=%(point1x)% > graphMaxX ? "adjust the right boundary by changing its value to higher than 1, then " : ""%><%=graphMinX > %(point1x)% ? "adjust the left boundary by lowering it to less than 1, then " : ""%><%=%(point1y)% > graphMaxY ? "adjust the upper boundary by increasing its value to more than 36, then " : ""%><%=graphMinY > %(point1y)% ? "change the lower boundary by decreasing it to less than 36, then " : ""%>add the point by clicking on the right spot in the graph.


In [11]:
# find the translation of the latest mass production table from the Greg's mass production table 
google_count = 0
google_dict = {}
for column in range(table_clean.shape[1]):
    for row in range(table_clean.shape[0]):
        # get the location
        column_name = table_clean.columns[column]
        column_num = table_translated.columns.get_loc(column_name)
        content_new = str(table_clean.iloc[row, column])
        if table_clean.index[row] == '%(startStateNodeName)%':
            # print("skip '%(startStateNodeName)%'")
            # write the english column
            table_translated[column_name].iloc[row] = content_new
            table_translated_clean[column_name].iloc[row] = content_new
            content_translated = content_new
            content_translated_clean = content_new
        # digit or empty, keep the original
        elif content_new.lstrip('-').replace(".", "").isdigit() or content_new == '':
            # print("digital or empty")
            table_translated[column_name].iloc[row] = content_new
            table_translated_clean[column_name].iloc[row] = content_new
            content_translated = content_new
            content_translated_clean = content_new
        # replace & translate
        else:
            # replace
            # judge whether a formula inside the content new
            pattern_formula = "<%(.*?)%>"
            # count the number of the replacement in one variable(content_new)
            match_formula = [match for match in re.findall(pattern_formula, str(content_new))]
            if match_formula:
                content_new, replacement_formula_dic = replace_formula(content_new, column)
            else:
                replacement_formula_dic = None
            content_new = replace_variable(content_new, column)
            if bool(replacement_formula_dic) is False:
                pass
            else:
                for key, value in replacement_formula_dic.items():
                    content_new = content_new.replace(key, value)
            table_translated[column_name].iloc[row] = content_new
            table_translated_clean[column_name].iloc[row] = content_new
            # translate
            # search in dict
            if content_new in google_dict:
                # print("find translation in dict")
                content_translated = '[google]' + google_dict[content_new]
                content_translated_clean = google_dict[content_new]
            # translate in google
            elif table_old.columns[(table_old == content_new).any()].empty:
                # print("use google translation")
                print(content_new)
                if match_formula:
                    content_new, translation_formula_dic = translate_formula(content_new)
                else:
                    translation_formula_dic = None
                try:
                    translation = tss.google(content_new, from_language='en', to_language=TARGET_LANG)
                except:
                    translation = 'error'
                print(content_new, translation_formula_dic)
                print(translation)
                if bool(translation_formula_dic) is False:
                    pass
                else:
                    for key, value in translation_formula_dic.items():
                        content_new = content_new.replace(key, value)
                        translation = translation.replace(key, value)
                content_translated = '[google]' + translation
                content_translated_clean = translation
                google_dict[content_new] = translation 
            # find translation in old table
            else:
                # print("find translation in sheet")
                column_name_old = table_old.columns[(table_old == content_new).any()][0]
                column_num_old = table_old.columns.get_loc(column_name_old)
                content_translated = table_old[table_old[column_name_old] == content_new].iloc[0, column_num_old+1]
                content_translated_clean = table_old[table_old[column_name_old] == content_new].iloc[0, column_num_old+1]    
        table_translated.iloc[row, column_num+1] = content_translated
        table_translated_clean.iloc[row, column_num+1] = content_translated_clean
# table_translated

Show All Feedback
Show All Feedback None
Mostrar todos los comentarios
You standing in line to get some frozen yogurt. There are a lot of people lining up.
You standing in line to get some frozen yogurt. There are a lot of people lining up. None
Estás en la fila para obtener un yogurt congelado. Hay mucha gente alineada.
You count that there are 42 people in front of you. The cashier helps 6 customers every minute.
You count that there are 42 people in front of you. The cashier helps 6 customers every minute. None
Usted considera que hay 42 personas frente a usted. El cajero ayuda a 6 clientes cada minuto.
Now, use the line you have drawn to answer 4 and 5 below.
Now, use the line you have drawn to answer 4 and 5 below. None
Ahora, use la línea que ha dibujado para responder 4 y 5 a continuación.
Read the introduction and think of names for the quantities and units. Fill them out in both the table and graph axes.
Read the introduction and think of names for the quantities and units. Fi

In [ ]:
print(google_count)

0


In [ ]:
# export the csv
table_translated_output = table_translated
# table_translated_output = table_translated.fillna("[CTAT]NAN")
table_translated_output.to_csv(path_output, encoding="utf-8", sep="\t")
table_translated_clean.to_csv(path_no_mark, encoding="utf-8", sep="\t")